In [11]:
import dlib
import os
from PIL import Image
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

In [4]:
import zipfile
path = '/content/drive/MyDrive/Datasets/yalefaces.zip'
zip_object = zipfile.ZipFile(file=path, mode = 'r')
zip_object.extractall('./')
zip_object.close()

In [2]:
face_detector = dlib.get_frontal_face_detector()
points_detector = dlib.shape_predictor('/content/drive/MyDrive/Weights/shape_predictor_68_face_landmarks.dat')
face_descriptor_extractor = dlib.face_recognition_model_v1('/content/drive/MyDrive/Weights/dlib_face_recognition_resnet_model_v1.dat')

In [18]:
index = {}
idx=0
face_descriptors = None

paths = [os.path.join('/content/yalefaces/train',f) for f in os.listdir('/content/yalefaces/train')]
for path in paths:
  # print(path)
  image = Image.open(path).convert('RGB')
  image_np = np.array(image,'uint8')
  face_detection = face_detector(image_np,1)
  for face in face_detection:
    l,t,r,b = face.left(), face.top(), face.right(), face.bottom()
    cv2.rectangle(image_np,(l,t),(r,b),(0,0,255),2)
    points = points_detector(image_np,face)
    for point in points.parts():
      cv2.circle(image_np,(point.x, point.y),2,(0,255,0),1)
    face_descriptor = face_descriptor_extractor.compute_face_descriptor(image_np,points)
    face_descriptor = [f for f in face_descriptor]
    face_descriptor = np.array(face_descriptor,dtype=np.float64)
    face_descriptor = face_descriptor[np.newaxis, :]
    # print(face_descriptor)
    if face_descriptors is None:
      face_descriptors = face_descriptor
    else:
      face_descriptors = np.concatenate((face_descriptors,face_descriptor),axis=0)

    index[idx] = path 
    idx += 1      

  # cv2_imshow(image_np)  


In [19]:
face_descriptors.shape

(132, 128)

In [20]:
face_descriptors

array([[-0.15016598,  0.08190971, -0.00158949, ..., -0.02205639,
         0.11158851,  0.05841742],
       [-0.16750331,  0.04092948,  0.07239178, ...,  0.02982324,
         0.11341026,  0.05801933],
       [-0.09758417,  0.01798972, -0.09853616, ..., -0.00577495,
        -0.01994056,  0.06262899],
       ...,
       [-0.1402103 ,  0.07552125,  0.01510925, ...,  0.01090842,
         0.06845607,  0.0286531 ],
       [-0.20720418,  0.09235502,  0.05147153, ..., -0.03726506,
         0.0900088 ,  0.06165224],
       [-0.12021624,  0.11514573,  0.08716735, ..., -0.02592804,
         0.18188372,  0.00609227]])